In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
import os
from torch.utils.data import DataLoader , ConcatDataset
from torchvision import datasets, transforms
import cv2
import sys
import math
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.2482, 0.2485, 0.2486], std=[0.2164, 0.2165, 0.2168]),
])

In [ ]:
tumor = []
healthy = []
for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/Tumor data set/brain_tumor_dataset/no/*.jpg"):
    img = cv2.imread(f)
    img = cv2.resize(img,(128,128))
    b, g, r = cv2.split(img)
    img = cv2.merge([r,g,b])
    tumor.append(img)

for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/Tumor data set/brain_tumor_dataset/yes/*.jpg"):
    img = cv2.imread(f)
    img = cv2.resize(img,(128,128))
    b, g, r = cv2.split(img)
    img = cv2.merge([r,g,b])
    healthy.append(img)

In [9]:
tumor=np.array(tumor,dtype=np.float32)
tumor.shape

(91, 128, 128, 3)

In [10]:
class Dataset(object):
    """An abstract class representing a Dataset.

    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

In [11]:
import cv2
import glob
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class MRI(Dataset):
    def __init__(self, transform=None):
        self.transform = transform

        tumor = []
        healthy = []

        # Load and preprocess images
        for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/Tumor data set/brain_tumor_dataset/no/*.jpg"):
            img = cv2.imread(f)
            img = cv2.resize(img, (128, 128))
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])
            img = img.transpose((2, 0, 1))  # Convert to (C, H, W)
            tumor.append(img)

        for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/Tumor data set/brain_tumor_dataset/yes/*.jpg"):
            img = cv2.imread(f)
            img = cv2.resize(img, (128, 128))
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])
            img = img.transpose((2, 0, 1))  # Convert to (C, H, W)
            healthy.append(img)

        # Concatenate and create labels
        tumor = np.array(tumor, dtype=np.float32)
        healthy = np.array(healthy, dtype=np.float32)

        tumor_label = np.ones(tumor.shape[0], dtype=np.float32)
        healthy_label = np.zeros(healthy.shape[0], dtype=np.float32)

        self.images = np.concatenate((tumor, healthy), axis=0)
        self.labels = np.concatenate((tumor_label, healthy_label))

        # Normalize images
        self.images /= 255.0

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = torch.tensor(self.images[index], dtype=torch.float32)
        label = torch.tensor(self.labels[index], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        sample = {'image': image, 'label': label}
        return sample

    def normalize(self):
      self.images = self.images/255.0


In [12]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn_model = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=5),
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=5))

        self.fc_model = nn.Sequential(
        nn.Linear(in_features=256, out_features=120),
        nn.Tanh(),
        nn.Linear(in_features=120, out_features=84),
        nn.Tanh(),
        nn.Linear(in_features=84, out_features=1))

    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        x = F.sigmoid(x)

        return x

In [13]:
mri_dataset = MRI()
mri_dataset.normalize()
device = torch.device('cuda:0')
model = CNN().to(device)


In [14]:
names={0:'Heathy', 1:'Tumor'}
dataloader = DataLoader(mri_dataset, batch_size=32, shuffle=False)

In [15]:
model.eval()
outputs = []
y_true = []
with torch.no_grad():
    for D in dataloader:
        image = D['image'].to(device)
        label = D['label'].to(device)

        y_hat = model(image)

        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())


In [16]:
outputs = np.concatenate( outputs, axis=0 ).squeeze()
y_true = np.concatenate( y_true, axis=0 ).squeeze()

In [17]:
def threshold(scores,threshold=0.50, minimum=0, maximum = 1.0):
    x = np.array(list(scores))
    x[x >= threshold] = maximum
    x[x < threshold] = minimum
    return x

In [18]:
accuracy_score(y_true, threshold(outputs))

0.6285714285714286

In [19]:
eta = 0.0001
EPOCH = 400
optimizer = torch.optim.Adam(model.parameters(), lr=eta)
dataloader = DataLoader(mri_dataset, batch_size=32, shuffle=True)
model.train()

CNN(
  (cnn_model): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=5, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=5, padding=0)
  )
  (fc_model): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Tanh()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Tanh()
    (4): Linear(in_features=84, out_features=1, bias=True)
  )
)

In [20]:
for epoch in range(1, EPOCH):
    losses = []
    for D in dataloader:
        optimizer.zero_grad()
        data = D['image'].to(device)
        label = D['label'].to(device)
        y_hat = model(data)
        # define loss function
        error = nn.BCELoss()
        loss = torch.sum(error(y_hat.squeeze(), label))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    if (epoch+1) % 10 == 0:
        print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch+1, np.mean(losses)))

Train Epoch: 10	Loss: 0.659006
Train Epoch: 20	Loss: 0.662076
Train Epoch: 30	Loss: 0.660911
Train Epoch: 40	Loss: 0.661750
Train Epoch: 50	Loss: 0.657280
Train Epoch: 60	Loss: 0.661800
Train Epoch: 70	Loss: 0.665790
Train Epoch: 80	Loss: 0.658057
Train Epoch: 90	Loss: 0.660905
Train Epoch: 100	Loss: 0.662049
Train Epoch: 110	Loss: 0.657054
Train Epoch: 120	Loss: 0.657790
Train Epoch: 130	Loss: 0.657777
Train Epoch: 140	Loss: 0.653662
Train Epoch: 150	Loss: 0.652118
Train Epoch: 160	Loss: 0.651998
Train Epoch: 170	Loss: 0.633137
Train Epoch: 180	Loss: 0.608866
Train Epoch: 190	Loss: 0.569510
Train Epoch: 200	Loss: 0.559355
Train Epoch: 210	Loss: 0.546703
Train Epoch: 220	Loss: 0.546133
Train Epoch: 230	Loss: 0.539539
Train Epoch: 240	Loss: 0.536275
Train Epoch: 250	Loss: 0.526155
Train Epoch: 260	Loss: 0.523351
Train Epoch: 270	Loss: 0.518249
Train Epoch: 280	Loss: 0.520092
Train Epoch: 290	Loss: 0.523763
Train Epoch: 300	Loss: 0.516603
Train Epoch: 310	Loss: 0.516692
Train Epoch: 320	

In [21]:
model.eval()
dataloader = DataLoader(mri_dataset, batch_size=32, shuffle=False)
outputs=[]
y_true = []
with torch.no_grad():
    for D in dataloader:
        image =  D['image'].to(device)
        label = D['label'].to(device)

        y_hat = model(image)

        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())

outputs = np.concatenate( outputs, axis=0 )
y_true = np.concatenate( y_true, axis=0 )

In [22]:
accuracy_score(y_true, threshold(outputs))

0.763265306122449